In [1]:
import numpy as np
import netwin as nw
import os
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
g = nx.gnp_random_graph(5, 0.5)
A = nx.adjacency_matrix(g)
D = np.diag([g.degree[i] for i in range(5)])
L = D - A
L = np.array(L)

In [3]:
def network_diffusion(p, t, theta):
    L, k = theta
    du = k * (np.matmul(-L, p))
    return du

def solve(f, p, t, theta):
    return odeint(f, p, t, args=(theta,))

def forward(f, u0, t, params): 
    p = u0[:-1]
    theta = params, u0[-1]
    #
    # print(u0)
    u = solve(f, p, t, theta) 

    return u.T 

In [4]:
    def setpriors(init_means):

        m0 = np.zeros_like(init_means)
        p0 = np.linalg.inv(np.diag(np.ones_like(m0) * 1e5))

        beta_mean0 = 1.0
        beta_var0  = 1000.0

        c0 = beta_var0 / beta_mean0
        s0 = beta_mean0**2 / beta_var0

        priors = m0, p0, c0, s0

        return priors

    def initialise(init_means, priors):
        if priors == None:
            priors = setpriors(init_means)
        
        m = init_means
        p = np.linalg.inv(np.diag(np.ones_like(m)))
        #c = np.array([priors[2]])
        #s = np.array([priors[3]])
        c = np.array([1e-8])
        s = np.array([50.0])
        params = m, p, c, s
        
        return params, priors

In [5]:
""" Scipt containing functions to implement variational inference using 
multivariate normal distribution and gamma distribution
"""

import numpy as np

def time_step(theta): 
    """Control time stepping interval proportional to the parameter magnitiude. 

    args: 
    theta:: float 
    """

    delta = theta * 1e-5
    if delta.any() < 0:
            delta = -delta
    if delta.any() < 1e-10:
            delta = 1e-10
    
    return delta

def central_difference(f, i, theta, delta, t):
    """ Calculate the derivate using a first order central difference approximation

    args:
        d : function 
            function on which to compute central difference 
        i : int 
            parameter index
    theta : np.array float 
            array of paramters for function, f
    delta : float 
            time-step for central difference
        t : np.array float
            time steps to evaluate at
    returns: 
        df : np.array, float 
             first order approximation, df, for time steps, t
    """
    dtheta = np.array(theta), np.array(theta)
    dtheta[0][i] += delta
    dtheta[1][i] -= delta
    f_1 = f(f=network_diffusion, u0=dtheta[0], t=t, params=L) 
    f_2 = f(f=network_diffusion, u0=dtheta[1], t=t, params=L)
    den = (2 * delta)
    df = (f_1 - f_2) / den
    return df

def Jacobian(f, theta, t):
    """Compute the Jacobian for globally defined function, f, with parameter set Theta 

    args:
    theta : np.array, float
            parameters of global function, f
        t : np.array, float 
            time steps to evaluate at

    returns: 
        J : np.array, float
            Jacobian vector/matrix evaluated at theta, t. 
    """
    J = None
    #f_n = len(f)
    p_n = len(theta)

    for i in range(p_n):
        delta = time_step(theta[i])
        if J is None:
                J = np.zeros([len(theta[:-1]) * len(t), len(theta)], dtype=np.float32)
        df = central_difference(f, i, theta, delta, t)
        J[:,i] = df.flatten()
    
    return J

def parameter_update(error, params, priors, J):
    """ Update forward model function parameters theta in accordance with the update equations above
    
    args: 
        error : array, float
                vector of the difference between observations and model prediction 
       params : tuple
                parameters values
       priors : tuple 
                priors values
            J : array, float 
                array of Jacbian values from the model given parameter updates (calculated above)
    returns: 
       params : tuple
                updated parameters values
    """
    m, p, s, c = params
    m0, p0, _, _ = priors

    p_new = s*c*np.dot(J.transpose(), J) + p0
    c_new = np.linalg.inv(p_new)
    print(error.shape)
    m_new = np.dot(c_new, (s * c * np.dot(J.transpose(), (error.flatten() +    np.dot(J, m))) + np.dot(p0, m0)))
    
    params[0][:], params[1][:] = m_new, p_new

    return params

def noise_update(error, data, params, priors, J):
    """ Update forward model function parameters phi in accordance with the update equations above
    
    args: 
        error : array, float
                vector of the difference between observations and model prediction 
       params : tuple
                parameters values
       priors : tuple 
                priors values
            J : array, float 
                array of Jacbian values from the model given parameter updates (calculated above)
    returns: 
       params : tuple
                updated parameters values
    """
    _, p, _, _ = params 
    _, _, s0, c0 = priors

    N = len(data)
    c = np.linalg.inv(p)
    c_new = N/2 + c0
    s_new = 1/(1/s0 + 1/2 * np.dot(error.flatten().transpose(), error.flatten()) + 1/2 * np.trace(np.dot(c, np.dot(J.transpose(), J))))
    
    params[2][:], params[3][:] = c_new, s_new

    return params

def error_update(y, f, theta, t):
    """Calculate difference between data and model with updated parameters

    args:
        y : array, float 
            vector of noisy data (observations)
    theta : array, float
            vector of parameter values for model, f
        t : array, float 
            vector of time steps at which to evaluate model 
    
    returns:
    error : array, float 
            vector of difference between noisy data and updated model
    """

    error = y - f(f=network_diffusion, u0=theta[0], t=t, params=L)
    
    return error

def fit(f, data, params, priors, t, n): 
    
    #theta = np.zeros((n, len(params[0])))

    for i in range(n):
        #theta[i,:] = params[0]

        error = error_update(data, f, params, t)

        J = Jacobian(f, params[0], t)
        print(J.shape)
        params = parameter_update(error, params, priors, J)
        params = noise_update(error, data, params, priors, J)
        print(params[0])
    return params

In [6]:
p = np.zeros([5])
p[2] = 20
k = 5

u0 = np.append(np.exp(p), k)
params = L

t = np.linspace(0,1,100)

p0 = np.ones([5])
k0 = 1

u_0 = np.append(p0, k0)

In [7]:
u = forward(f=network_diffusion,u0=u0,t=t,params=params)

In [8]:
params, priors = initialise(init_means=u_0, priors=None)
params[0]

array([1., 1., 1., 1., 1., 1.])

In [9]:
fi = fit(f=forward, data=u, params=params, priors=priors, t=t, n=500)

1.43628162  1.4362621   3.16429631]
(500, 6)
(5, 100)
[ 1.43626696  1.43626626 14.04958628  1.43627882  1.43625953  3.16429983]
(500, 6)
(5, 100)
[ 1.43627397  1.43626562 14.04957194  1.43627902  1.436267    3.16429314]
(500, 6)
(5, 100)
[ 1.43627196  1.43626707 14.04957757  1.43628042  1.43626208  3.16429779]
(500, 6)
(5, 100)
[ 1.43627192  1.43627101 14.04957627  1.43627748  1.43626058  3.16429395]
(500, 6)
(5, 100)
[ 1.43627424  1.4362636  14.04957994  1.4362773   1.43626368  3.16429703]
(500, 6)
(5, 100)
[ 1.43626926  1.4362683  14.04958404  1.43627449  1.43626151  3.1642992 ]
(500, 6)
(5, 100)
[ 1.43627276  1.43626488 14.04957298  1.43628509  1.43626169  3.16429306]
(500, 6)
(5, 100)
[ 1.43627249  1.43626707 14.04957488  1.43628189  1.43626268  3.16429619]
(500, 6)
(5, 100)
[ 1.4362675   1.43626644 14.04958477  1.43627931  1.43625966  3.16429881]
(500, 6)
(5, 100)
[ 1.43627073  1.43626949 14.04957453  1.43628343  1.43625945  3.16429435]
(500, 6)
(5, 100)
[ 1.43627401  1.43626266 1

In [10]:
fi

(array([ 1.43627236,  1.43626397, 14.04957156,  1.43628408,  1.43626583,
         3.16429283]),
 array([[ 2.40991238e-04,  1.85752481e-04,  1.85752554e-04,
          1.85752481e-04,  1.75750269e-04,  1.19204076e-05],
        [ 1.85752481e-04,  2.30989063e-04,  1.85752573e-04,
          1.85752481e-04,  1.85752536e-04,  1.19204397e-05],
        [ 1.85752554e-04,  1.85752573e-04,  2.30989100e-04,
          1.85752518e-04,  1.85752573e-04, -4.76817084e-05],
        [ 1.85752481e-04,  1.85752481e-04,  1.85752518e-04,
          2.30988989e-04,  1.85752536e-04,  1.19204041e-05],
        [ 1.75750269e-04,  1.85752536e-04,  1.85752573e-04,
          1.85752536e-04,  2.40991238e-04,  1.19204719e-05],
        [ 1.19204076e-05,  1.19204397e-05, -4.76817084e-05,
          1.19204041e-05,  1.19204719e-05,  2.01681036e-04]]),
 array([2.501]),
 array([3.85445504e-06]))